In [1]:
import os
import sys
current_folder = os.path.dirname(os.path.abspath('__file__'))
parent_folder = os.path.dirname(current_folder)
grandparent_folder = os.path.dirname(parent_folder)
grandgrandparent_folder = os.path.dirname(grandparent_folder)
grandgrandgrandparent_folder = os.path.dirname(grandgrandparent_folder)
sys.path.append(grandgrandgrandparent_folder)

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.DeepONet import DeepONet
from methods.methodsDataset.DeepONetDataset import DeepONetDataset
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 0., 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_DONUT ={
             'branch': {'layer_dims': None, 'activations': None, 'device': 'cpu', 'seed': 123}, 
             'trunk':{'layer_dims': None, 'activations': None, 'device': 'cpu', 'seed': 123}
             }

In [5]:
Dmin, Dmax = 0.1, 10
D_list = np.linspace(Dmin, Dmax, 1000).reshape(-1, 1)

U_sols = []
solver = Solver(params={'solver': params_solver})
for d in D_list:
    solver.change_D(new_D=d)
    U_sols.append(solver.solve())
U_sols = np.stack(U_sols)

In [6]:
d_train, d_val, u_train, u_val = sklearn.model_selection.train_test_split(D_list, U_sols, test_size=0.2, 
                                                                          random_state=params_DONUT['branch']['seed'])


In [7]:
def get_dataset():
    return d_train, d_val, u_train, u_val

In [8]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [17]:
def define_model(trial, input_size):
    nb_basis = trial.suggest_int("nb_basis", 8, 40)
    n_layers_branch = trial.suggest_int("n_layers_branch", 1, 5)
    branch_layers = [input_size]
    
    for i in range(n_layers_branch):
        out_features = trial.suggest_int("branch_units_l{}".format(i), 8, 40)
        branch_layers += [out_features]
    branch_layers += [nb_basis]

    n_layers_trunk = trial.suggest_int("n_layers_trunk", 1, 5)
    trunk_layers = [input_size]
    
    for i in range(n_layers_trunk):
        out_features = trial.suggest_int("trunk_units_l{}".format(i), 8, 40)
        trunk_layers += [out_features]
    trunk_layers += [nb_basis]
    
    params_DONUT_trial = copy.deepcopy(params_DONUT)
    params_DONUT_trial['branch']['layer_dims'] = branch_layers
    params_DONUT_trial['branch']['activations'] = 'tanh'
    params_DONUT_trial['trunk']['layer_dims'] = trunk_layers
    params_DONUT_trial['trunk']['activations'] = 'tanh'
 
    return DeepONet(params={'solver':params_solver, 'method':params_DONUT_trial})

In [27]:
def objective(trial):
    # Generate the model.
    model = define_model(trial, 1)

    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
 
    D_train, D_val, U_train, U_val = get_dataset()
    
    torch.manual_seed(params_DONUT['branch']['seed'])
    n_epochs = 10000
    hyperparameters = {'lr': lr, 'epochs': n_epochs, 'optimizer': 'Adam'}
    
    model.fit(hyperparameters=hyperparameters, 
              U_train=U_train, U_val=U_val, D_train=D_train, D_val=D_val)
        
    val_loss = model.loss_dict['val']
    trial.report(min(val_loss), n_epochs)
    return min(val_loss)

In [28]:
sampler = TPESampler(seed=params_DONUT['branch']['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-09 17:35:32,805] A new study created in memory with name: no-name-87aeab96-411f-4b8f-a494-662dbe8576a9


In [29]:
study.optimize(objective, n_trials=3)

[tr : 6.3e-05, val : 4.7e-05]: 100%|█████| 10000/10000 [00:21<00:00, 470.95it/s]
[I 2023-08-09 17:35:54,616] Trial 0 finished with value: 4.724414611700922e-05 and parameters: {'nb_basis': 30, 'n_layers_branch': 2, 'branch_units_l0': 15, 'branch_units_l1': 26, 'n_layers_trunk': 4, 'trunk_units_l0': 21, 'trunk_units_l1': 40, 'trunk_units_l2': 30, 'trunk_units_l3': 23, 'lr': 0.00015009027543233888}. Best is trial 0 with value: 4.724414611700922e-05.
[tr : 1.6e-05, val : 9.3e-06]: 100%|█████| 10000/10000 [00:20<00:00, 491.14it/s]
[I 2023-08-09 17:36:14,981] Trial 1 finished with value: 9.28167901292909e-06 and parameters: {'nb_basis': 19, 'n_layers_branch': 4, 'branch_units_l0': 22, 'branch_units_l1': 9, 'branch_units_l2': 21, 'branch_units_l3': 32, 'n_layers_trunk': 1, 'trunk_units_l0': 13, 'lr': 0.00039323677119884875}. Best is trial 1 with value: 9.28167901292909e-06.
[tr : 1.1e-05, val : 9.7e-06]: 100%|█████| 10000/10000 [00:21<00:00, 461.51it/s]
[I 2023-08-09 17:36:36,654] Trial 2 fi

In [30]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  9.28167901292909e-06
  Params: 
    nb_basis: 19
    n_layers_branch: 4
    branch_units_l0: 22
    branch_units_l1: 9
    branch_units_l2: 21
    branch_units_l3: 32
    n_layers_trunk: 1
    trunk_units_l0: 13
    lr: 0.00039323677119884875


In [37]:
branch_activations = []
branch_layer_dims = []

trunk_activations = []
trunk_layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'branch':
        if key.split('_')[1] == 'activation':
            branch_activations.append(value)
        elif key.split('_')[1] == 'units':
            branch_layer_dims.append(value)
        
    elif key.split('_')[0] == 'trunk':
        if key.split('_')[1] == 'activation':
            trunk_activations.append(value)
        elif key.split('_')[1] == 'units':
            trunk_layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value
    elif key == 'batch_size':
        batch_size = value
    elif key == 'nb_basis':
        basis_number = value

In [38]:
print('branch:\n', branch_activations, '\n', branch_layer_dims, 
      '\ntrunk:\n', trunk_activations, '\n', trunk_layer_dims,
      '\nhyperparameters\n',
      '\n', lr, '\n',  '\n', basis_number)

branch:
 [] 
 [22, 9, 21, 32] 
trunk:
 [] 
 [13] 
hyperparameters
 
 0.00039323677119884875 
 
 19
